In [ ]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>

# <h1><center><ins>PICKLE y FLASK</ins></center></h1>
<h1><center>Checkpoint [API]  —solución—:</center></h1>
<img src="img/00_pickle.jpg" alt="Drawing" style="width: 400px;"/>
<img src="img/04_flask.jpg" alt="Drawing" style="width: 500px;"/>

<a id="tabla_contenidos"></a> 
## Tabla de Contenidos

### <a href='#section_objetivos'>0. Objetivos de la Notebook</a>


### <a href='#section_repaso'>1. Lo que tenés que hacer es...</a>

<a id="section_objetivos"></a> 
## 0. Objetivos de la Notebook

<div id="caja11" style="float:left;width: 100%;">
  <div style="float:left;width: 9%;"><img src="../../../common/icons/haciendo_foco.png" style="align:left"/> </div>
  <br>
  <div style="float:left;width: 85%;">
      <label>Poner en práctica lo aprendido en la notebook de práctica guiada.</label>
  <div style="float:left;width: 85%;">
      <label>Es importante que antes de la clase resuelvan esta notebook ya que es fundamental que sepan utilizar las herramientas que vimos en la práctica guiada para después trabajar el caso práctico en la clase.</label>        
</div>    
</div>

<a href='#tabla_contenidos'>Volver a TOC</a>

***

<a id="section_repaso"></a> 
## 1. Lo que tenés que hacer es...

El objetivo de esta práctica es simular nuestro propio servidor para poner a disposición nuestra API para un cliente interno. 

Vamos a utilizar un set de datos adaptados de una competencia de kaggle ya cerrada: https://www.kaggle.com/kevinmh/fifa-18-more-complete-player-dataset

En el dataset vamos a encontrar todos los jugadores de fútbol que están en el juego fifa-18, y contaremos con las siguientes variables: 
- **ID**: un número único que identifica al jugador en toda la base.
- **full_name**: nombre completo del jugador.
- **age**
- **club**: del jugador
- **height_cm**
- **weight_kg**
- **puntaje_global**: puntaje que identifica la habilidad general del jugador.
- **potencia**: potencia física del jugador.
- **ritmo**: velocidad de aceleración del jugador.
- **disparos**: nivel de precisión y potencia de sus remates.
- **pases**: nivel de precisión en sus pases.
- **amagues**: nivel de habilidad para amagar a un rival.
- **defensa**: capacidad defensiva general del jugador.
- **físico**: estado físico del jugador (nos indicaría qué tan rápido se cansa)

Su **misión** es simular en esta notebook un **SERVIDOR** donde está alojada su API y, desde otra notebook (Checkpoint_Pickle_shelve_flask_CLIENT) hacer los pedidos simulando que son un **CLIENTE**.

Su API va a hacer dos cosas (con dos funciones/endpoints distintos): 
- recibir el dataframe completo con todos los jugadores y quedarse con aquellos que pertenecen a un club específico.
- hacer una comparación de dos jugadores del mismo equipo.

<div id="caja9" style="float:left;width: 100%;">
  <div style="float:left;width: 15%;"><img src="../../../common/icons/kit_de_salida.png" style="align:left"/> </div>
  <br>
  <div style="float:left;width: 85%;"><label><b>Para tener en cuenta:</b> Tal como vimos en las notebooks de la práctica guiada, por un lado tienen que definir las funionces/endpoints de su API y, al final de la notebook, ejecutar la línea de código <b>app.run(host='0.0.0.0')</b>. Recuerden que esta línea hace que la notebook simule ser un servidor que está poniendo a disposición a nuestra API para que podamos acceder desde la otra notebook que es el cliente (usando la librería request). Y por último: mientras esté en ejecución esta línea de código, acuérdense de que no pueden ejecutar ninguna otra celda de esta notebook. </label></div>
</div>

Importamos las librerías que vamos a utilizar

In [ ]:
# Algunas de las de siempre
import numpy as np
import pandas as pd

# Las que vamos a necesitar para hacer los llamados y para lidiar con los datos en formato json
import requests
from flask import  Flask, request, jsonify, render_template
import json
import pickle

# Librerías y configuración para visualizaciones
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_context("talk")
sns.set_style('darkgrid')
sns.set_context(rc={'xtick.labelsize': 13,'ytick.labelsize': 13, 'axes.labelsize': 16, 'axes.titlesize': 18})
# Esta función la vamos a utilizar para hacer un radar-chart
from math import pi


***

## **PRIMERA FUNCIÓN/ENDPOINT:** filtrar por club, guardar nuevo dataframe y figura

> La primera función/endpoint tiene que recibir, por lo menos:
- (i) el dataframe con la base de jugadores completos "fifa_2018_completo_editado.csv" que se encuentra dentro de la carpeta Data (tienen que pasar la base como un json), 
- (ii) el club por el cual quieren filtrar la base completa

> El **output** de esta primera función/endpoint tiene que ser: 
- (i) un json que contenga la cantidad de jugadores del equipo elegido, la media de la edad, el peso y la altura de los jugadores del equipo, y la lista completa de jugadores del equipo.
- (ii) guardar el dataframe filtrado con los jugadores del club elegido usando la librería pickle.
- (iii) guardar una figura (puede ser un barplot) que contenga la media del equipo entre todos los jugadores de las siguientes features: ['potencia','ritmo','disparos','pases','amagues','defensa','fisico'].

In [ ]:
# Iniciamos nuestra API
app = Flask('Analisis equipo')

In [ ]:
# Iniciamos nuestro primera función asociada a su endpoint. Usamos el método POST ya que vamos a enviar información
# al servidor: el dataframe filtrado y vamos a guardar un radar chart con la media de las características más 
# importantes del equipo

In [ ]:
@app.route('/describir_equipo',methods=['POST'])
def descriptivos_equipo():
    # la función "request.get_json" de Flask para capturar la información que le envíemos a la API
    data = request.get_json(force=True)
    
    # Separamos la información de data y usamos json.loads() para transformar el dataframe que está en formato
    # json a un diccionario y luego lo convertimos en un DataFrame.
    df=pd.DataFrame(json.loads(data['base']))
    club=data['club_elegido']
    color=data['radar_color']
    size=data['fig_size']
    
    
    # Filtramos el dataframe para quedarnos con los jugadores pertenecientes al club elegido por el usuario
    df_club=df[df['club']==club]
    # Guardamos la base filtrada con el nombre del club usando PICKLE en la carpeta Data
    with open('../Data/'+club+'.pkl', 'wb') as df_pkl:
        pickle.dump(df_club, df_pkl)
    
    
    
    # Vamos a guardar un radar chart. En la siguiente página pueden encontrar un ejemplo y los detalles para
    # hacerlo: https://python-graph-gallery.com/390-basic-radar-chart/
    # Primero estimamos la media de las características del club que nos interesan
    df_to_plot=pd.DataFrame(df_club[['potencia','ritmo','disparos','pases','amagues','defensa','fisico']].mean()).T
    
    #Definimos una serie de variables auxiliares para hacer el radar-chart
    # number of variable
    categories=list(df_to_plot)
    N = len(categories)
    # We are going to plot the first line of the data frame.
    # But we need to repeat the first value to close the circular graph:
    values=df_to_plot.loc[0].values.flatten().tolist()
    values += values[:1]
    # What will be the angle of each axis in the plot? (we divide the plot / number of variable)
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1]
    # Initialise the spider plot
    fig, ((ax)) = plt.subplots(1,1,gridspec_kw={'hspace': 0.45, 'wspace': 0.15},figsize=size,dpi=300)
    fig.suptitle(club,y=0.96,x=0.2,fontsize=30,fontweight='bold')
    ax = plt.subplot(111, polar=True)
    # Draw one axe per variable + add labels labels yet
    plt.xticks(angles[:-1], categories, color='black', size=20)
    # Draw ylabels
    ax.set_rlabel_position(0)
    plt.yticks([20,30,40,50,60,70,80,90], ["20","30","40","50","60","70","80","90"], color="grey", size=18)
    plt.ylim(10,100)
    # Plot data
    ax.plot(angles, values, linewidth=2, linestyle='solid')
    # Fill area
    ax.fill(angles, values, color, alpha=0.2)
    plt.close(); #usamos esta línea para que la figura no se imprima en pantalla
    # guardamos la figura a la altura en la que se encuentran las notebooks usando el nombre del club
    fig.savefig(club+'.jpg',dpi=150)
    
    
    # Armamos el return con los datos descriptivos del equipo seleccionado que no están en el gráfico

    return {'cant_jugadores':int(df_club.shape[0]),
            'media_altura':float(df_club['height_cm'].mean()),
            'media_peso':float(df_club['weight_kg'].mean()),
            'media_edad':float(df_club['age'].mean()),
            'lista_jugadores':list(df_club['name'].unique())}
    

***

## **SEGUNDA FUNCIÓN/ENDPOINT:** comparar jugadores del club elegido
> La segunda función/endpoint tiene que recibir, por lo menos:
- (i) el nombre del dataframe filtrado que se guardó como un pickle con la función anterior.
- (ii) nombre de los dos jugadores que se quieren comparar.

> El **output** de esta primera función/endpoint tiene que ser: 
- (i) un json que contenga las características de cada uno de los jugadores elegidos en las siguientes features: ['potencia','ritmo','disparos','pases','amagues','defensa','fisico']
- (ii) guardar una figura (puede ser un barplot) que compare la media entre los dos jugadores de las siguientes features: ['potencia','ritmo','disparos','pases','amagues','defensa','fisico']

In [ ]:
# Iniciamos nuestro primera función asociada a su endpoint. Usamos el método GET ya que no le vamos a enviar información
# al servidor, sino que le vamos a pedir información a partir de la base que se filtra en el primer endpoint

In [ ]:
@app.route("/comparar_jugadores",methods=['GET'])
def comparador():
    
    # Usamos request.args para tomar las query que le pasamos a la URL
    club=request.args['club']
    uno=request.args['jugador_uno']
    dos=request.args['jugador_dos']
    c_uno=request.args['color_uno']
    c_dos=request.args['color_dos']
    
    # Levantamos el dataframe que tenemos guardado en disco
    with open('../Data/'+club+'.pkl', 'rb') as df_pkl:
        df_sel = pickle.load(df_pkl)
        
    # Filatramos el dataframe para hacer el radar-chart
    df_to_plot=df_sel[['name','potencia','ritmo','disparos','pases','amagues','defensa','fisico']]
    df_to_plot=df_to_plot.reset_index().drop(columns='index')
    df_to_plot.head()
    
    # Generamos el radar-chart de la siguiente página: 
    # https://python-graph-gallery.com/391-radar-chart-with-several-individuals/
    # ------- PART 1: Create background
 
    # number of variable
    categories=list(df_to_plot.drop(columns='name'))
    N = len(categories)

    # What will be the angle of each axis in the plot? (we divide the plot / number of variable)
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1]
    
    # Initialise the spider plot
    fig, ((ax)) = plt.subplots(1,1,gridspec_kw={'hspace': 0.45, 'wspace': 0.15},figsize=(25,15),dpi=300)
    fig.suptitle(club,y=0.96,x=0.3,fontsize=30,fontweight='bold')

    # Initialise the spider plot
    ax = plt.subplot(111, polar=True)
    # If you want the first axis to be on top:
    ax.set_theta_offset(pi / 2)
    ax.set_theta_direction(-1)
    # Draw one axe per variable + add labels labels yet
    plt.xticks(angles[:-1], categories, color='black', size=20)
    # Draw ylabels
    ax.set_rlabel_position(0)
    plt.yticks([20,30,40,50,60,70,80,90], ["20","30","40","50","60","70","80","90"], color="grey", size=18)
    plt.ylim(10,100)


    # ------- PART 2: Add plots
    # Plot each individual = each line of the data
    # I don't do a loop, because plotting more than 3 groups makes the chart unreadable
    # Ind1
    values_uno=df_to_plot[df_to_plot['name']==uno].drop(columns='name').values.flatten().tolist()
    values_uno += values_uno[:1]
    ax.plot(angles, values_uno, linewidth=2, linestyle='solid', label=uno,c=c_uno)
    ax.fill(angles, values_uno, c_uno, alpha=0.2)
    # Ind2
    values=df_to_plot[df_to_plot['name']==dos].drop(columns='name').values.flatten().tolist()
    values += values[:1]
    ax.plot(angles, values, linewidth=2, linestyle='solid', label=dos,c=c_dos)
    ax.fill(angles, values, c_dos, alpha=0.2)
    # Add legend
    plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
    plt.close()
    fig.savefig(club+'_'+uno+'_vs_'+dos+'.jpg',dpi=150)
    
    # return datos individuales de los jugadores en un json
    return df_to_plot[df_to_plot['name'].isin([uno,dos])].set_index('name').to_json()
    

In [ ]:
# Ahora ejecutamos esta línea de código que pone a disposición los endpoints que armamos arriba. 
# Es hora de ir a la otra notebook en la que simulamos ser un cliente y hacer los llamadados para cada
# uno de estos endpoints...
app.run(host='0.0.0.0', port = 5008)